In [ ]:
import os
from uuid import uuid4

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate

from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance


# ----------------------------
# CONFIG
# ----------------------------
COLLECTION_NAME = "rag_demo"
QDRANT_URL = os.getenv("QDRANT_URL", "http://localhost:6333")

EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # 384 dims


# ----------------------------
# Prompt Template
# ----------------------------
rag_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Answer the question using ONLY the context below.\n"
        "If the answer is not present, say \"I don't know\".\n\n"
        "Context:\n{context}\n\n"
        "Question:\n{question}\n\n"
        "Answer:\n"
    ),
)


# ----------------------------
# Helpers
# ----------------------------
def load_pdf_documents(pdf_path: str):
    loader = PyPDFLoader(pdf_path)
    return loader.load()  # List[Document]


def chunk_documents(docs, chunk_size: int = 1000, chunk_overlap: int = 100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    return splitter.split_documents(docs)


def ensure_collection(client: QdrantClient, collection_name: str, vector_size: int):
    existing = {c.name for c in client.get_collections().collections}
    if collection_name not in existing:
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
        )


def build_vector_store(client: QdrantClient, collection_name: str, embeddings):
    return QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings,
    )


def build_llm():
    # Minimal params to avoid unsupported param errors on hosted endpoints.
    return HuggingFaceEndpoint(
        repo_id="Qwen/Qwen3-30B-A3B",
        max_new_tokens=512,
        huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    )


def insert_pdf(vector_store: QdrantVectorStore):
    pdf_path = input("Enter path to PDF file: ").strip()
    if not pdf_path:
        print("No path provided.")
        return

    docs = load_pdf_documents(pdf_path)

    # Attach source filename
    for d in docs:
        d.metadata = d.metadata or {}
        d.metadata["source"] = pdf_path

    chunks = chunk_documents(docs)

    ids = [str(uuid4()) for _ in chunks]
    vector_store.add_documents(documents=chunks, ids=ids)

    print(f" Inserted {len(chunks)} chunks into `{COLLECTION_NAME}`")


def fetch_context(vector_store: QdrantVectorStore, k: int = 5) -> str | None:
    query_text = input("Enter search query: ").strip()
    if not query_text:
        print("No query provided.")
        return None

    results = vector_store.similarity_search(query_text, k=k)
    if not results:
        print("No matching results found.")
        return None

    lines = []
    for r in results:
        src = r.metadata.get("source", "unknown")
        page = r.metadata.get("page", "n/a")
        lines.append(f"[source={src} page={page}]\n{r.page_content}")

    context = "\n\n---\n\n".join(lines)
    print("\n🔎 Retrieved Context (preview):\n", context[:800])
    return context


def chat_rag(llm, vector_store: QdrantVectorStore):
    context = fetch_context(vector_store)
    if not context:
        return

    question = input("\nEnter your question about the retrieved text: ").strip()
    if not question:
        print("No question provided.")
        return

    prompt = rag_prompt_template.format(context=context, question=question)

    answer = llm.invoke(prompt)
    print("\n🤖 AI Response:\n", answer)


def main():
    print("RAG Demo (Python 3.12) - PDF → Qdrant → Retrieve → LLM\n")

    embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

    # Auto-detect vector size
    vector_size = len(embeddings.embed_query("dimension check"))
    print(f"✅ Embedding model: {EMBED_MODEL}")
    print(f"✅ Vector size detected: {vector_size}\n")

    client = QdrantClient(url=QDRANT_URL)
    ensure_collection(client, COLLECTION_NAME, vector_size)

    vector_store = build_vector_store(client, COLLECTION_NAME, embeddings)
    llm = build_llm()

    while True:
        choice = input(
            "\nChoose an action:\n"
            "1. Insert Text from PDF\n"
            "2. Fetch & Interact with AI\n"
            "3. Exit\n"
            "Enter choice: "
        ).strip()

        if choice == "1":
            insert_pdf(vector_store)
        elif choice == "2":
            chat_rag(llm, vector_store)
        elif choice == "3":
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")


if __name__ == "__main__":
    main()

RAG Demo (Python 3.12) - PDF → Qdrant → Retrieve → LLM

✅ Embedding model: sentence-transformers/all-MiniLM-L6-v2
✅ Vector size detected: 384

 Inserted 94 chunks into `rag_demo`

🔎 Retrieved Context (preview):
 [source=/Users/anandh/projects/Agentic-AI-course/VectorDB/Americas-AI-Action-Plan.pdf page=5]
Federal funding to be directed toward states with burdensome AI regulations that waste these 
funds, but should also not interfere with states’ rights to pass prudent laws that are not unduly 
restrictive to innovation. 
Recommended Policy Actions 
• Led by the Office of Science and Technology Policy (OSTP), launch a Request for 
Information from businesses and the public at large about current Federal regulations 
that hinder AI innovation and adoption, and work with relevant Federal agencies to take 
appropriate action. 
• Led by the Office of Management and Budget (OMB) and c onsistent with Executive 
Order 14192 of January 31, 2025, “ Unleashing Prosperity Through Deregulation, ” w